In [1]:
import torch
import torch.nn as nn

X = torch.tensor([[1,2,3],
                  [4,5,6],
                  [7,8,9]], dtype=torch.float32)

# X = torch.unsqueeze(X, 0)
print(X.shape)
X

torch.Size([3, 3])


tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])

### 참고
PyTorch에서 dim은 텐서의 차원을 나타내며, 다음과 같이 이해할 수 있습니다:
1. dim=0: 첫 번째 차원, 일반적으로 배치(batch) 차원 또는 행(row)을 나타냅니다.
2. dim=1: 두 번째 차원, 보통 열(column)을 나타냅니다.
3. dim=2: 세 번째 차원, 3D 텐서에서는 깊이(depth)를 나타냅니다.

예를 들어, 2D 텐서 X에 대해:
```python
X = torch.tensor([[1, 2, 3],
                  [4, 5, 6],
                  [7, 8, 9]])
```
- torch.sum(X, dim=0)은 열 방향으로 합을 계산합니다: [12, 15, 18]
- torch.sum(X, dim=1)은 행 방향으로 합을 계산합니다: [6, 15, 24]

In [2]:
print(torch.sum(X, dim=0))
print(torch.sum(X, dim=1))

tensor([12., 15., 18.])
tensor([ 6., 15., 24.])


### BatchNorm

In [3]:
bn = nn.BatchNorm1d(3)  # 3은 특성(feature) 수
output_bn = bn(X)
print("Batch Normalization 결과:")
print(output_bn)

Batch Normalization 결과:
tensor([[-1.2247, -1.2247, -1.2247],
        [ 0.0000,  0.0000,  0.0000],
        [ 1.2247,  1.2247,  1.2247]], grad_fn=<NativeBatchNormBackward0>)


In [5]:
print(torch.mean(X, dim=0))
print(torch.std(X, dim=0))

tensor([4., 5., 6.])
tensor([3., 3., 3.])


In [72]:
def batch_norm(X, eps=1e-5):
    mean = X.mean(dim=0, keepdim=True)
    var = X.var(dim=0, unbiased=False, keepdim=True)
    X_norm = (X - mean) / torch.sqrt(var + eps)
    return X_norm

print("Batch Normalization 결과:")
print(batch_norm(X))

Batch Normalization 결과:
tensor([[-1.2247, -1.2247, -1.2247],
        [ 0.0000,  0.0000,  0.0000],
        [ 1.2247,  1.2247,  1.2247]])


### Layer Norm

In [73]:
ln = nn.LayerNorm(3)  # 3은 정규화할 마지막 차원의 크기
output_ln = ln(X)
print("Layer Normalization 결과:")
print(output_ln)

Layer Normalization 결과:
tensor([[-1.2247,  0.0000,  1.2247],
        [-1.2247,  0.0000,  1.2247],
        [-1.2247,  0.0000,  1.2247]], grad_fn=<NativeLayerNormBackward0>)


In [74]:
def layer_norm(X, eps=1e-5):
    mean = X.mean(dim=-1, keepdim=True)
    var = X.var(dim=-1, unbiased=False, keepdim=True)
    X_norm = (X - mean) / torch.sqrt(var + eps)
    return X_norm

print("Layer Normalization 결과:")
print(layer_norm(X))

Layer Normalization 결과:
tensor([[-1.2247,  0.0000,  1.2247],
        [-1.2247,  0.0000,  1.2247],
        [-1.2247,  0.0000,  1.2247]])


### RMS Norm

In [64]:
class RMSNorm(nn.Module):
    def __init__(self, dim, eps=1e-8):
        super().__init__()
        self.scale = nn.Parameter(torch.ones(dim))
        self.eps = eps

    def forward(self, x):
        rms = torch.sqrt(torch.mean(x**2, dim=-1, keepdim=True) + self.eps)
        x_norm = x / rms
        return self.scale * x_norm

rms_norm = RMSNorm(3)  # 3은 특성(feature) 수
output_rms = rms_norm(X)
print("RMS Normalization 결과:")
print(output_rms)

RMS Normalization 결과:
tensor([[0.4629, 0.9258, 1.3887],
        [0.7895, 0.9869, 1.1843],
        [0.8705, 0.9948, 1.1192]], grad_fn=<MulBackward0>)


In [65]:
def rms_norm(X, eps=1e-8):
    rms = torch.sqrt(torch.mean(X**2, dim=-1, keepdim=True) + eps)
    X_norm = X / rms
    return X_norm

print("RMS Normalization 결과:")
print(rms_norm(X))

RMS Normalization 결과:
tensor([[0.4629, 0.9258, 1.3887],
        [0.7895, 0.9869, 1.1843],
        [0.8705, 0.9948, 1.1192]])


### Instance Norm

In [6]:
# 입력 (2개의 샘플, 각 2x2 크기, 4 채널)
X = torch.tensor([[
                  [[1, 2], [3, 4]],
                  [[5, 6], [7, 8]],
                  [[9, 10], [11, 12]],
                  [[13, 14], [15, 16]]
                  ],
                  [
                  [[17, 18], [19, 20]],
                  [[21, 22], [23, 24]],
                  [[25, 26], [27, 28]],
                  [[29, 30], [31, 32]]
                  ]], dtype=torch.float32)
print(X.shape)
X

torch.Size([2, 4, 2, 2])


tensor([[[[ 1.,  2.],
          [ 3.,  4.]],

         [[ 5.,  6.],
          [ 7.,  8.]],

         [[ 9., 10.],
          [11., 12.]],

         [[13., 14.],
          [15., 16.]]],


        [[[17., 18.],
          [19., 20.]],

         [[21., 22.],
          [23., 24.]],

         [[25., 26.],
          [27., 28.]],

         [[29., 30.],
          [31., 32.]]]])

In [7]:
# Instance Normalization은 주로 2D나 3D 데이터에 사용되지만, 
# InstanceNorm2d 적용 (4은 채널 수)
in_norm = nn.InstanceNorm2d(4, affine=True)
output_in = in_norm(X)
print("Instance Normalization 결과 (PyTorch):")
print(output_in)

Instance Normalization 결과 (PyTorch):
tensor([[[[-1.3416, -0.4472],
          [ 0.4472,  1.3416]],

         [[-1.3416, -0.4472],
          [ 0.4472,  1.3416]],

         [[-1.3416, -0.4472],
          [ 0.4472,  1.3416]],

         [[-1.3416, -0.4472],
          [ 0.4472,  1.3416]]],


        [[[-1.3416, -0.4472],
          [ 0.4472,  1.3416]],

         [[-1.3416, -0.4472],
          [ 0.4472,  1.3416]],

         [[-1.3416, -0.4472],
          [ 0.4472,  1.3416]],

         [[-1.3416, -0.4472],
          [ 0.4472,  1.3416]]]], grad_fn=<ViewBackward0>)


In [9]:
def instance_norm(X, eps=1e-5):
    mean = X.mean(dim=(2, 3), keepdim=True)
    var = X.var(dim=(2, 3), unbiased=False, keepdim=True)
    X_norm = (X - mean) / torch.sqrt(var + eps)
    return X_norm

print("Instance Normalization 결과 (수동 구현):")
print(instance_norm(X))

Instance Normalization 결과 (수동 구현):
tensor([[[[-1.3416, -0.4472],
          [ 0.4472,  1.3416]],

         [[-1.3416, -0.4472],
          [ 0.4472,  1.3416]],

         [[-1.3416, -0.4472],
          [ 0.4472,  1.3416]],

         [[-1.3416, -0.4472],
          [ 0.4472,  1.3416]]],


        [[[-1.3416, -0.4472],
          [ 0.4472,  1.3416]],

         [[-1.3416, -0.4472],
          [ 0.4472,  1.3416]],

         [[-1.3416, -0.4472],
          [ 0.4472,  1.3416]],

         [[-1.3416, -0.4472],
          [ 0.4472,  1.3416]]]])


### Group Norm

In [10]:
import torch
import torch.nn as nn

def group_norm_pytorch(X, num_groups=2):  # 그룹 수를 2으로 변경
    # num_groups: 그룹의 수
    # num_channels: 채널의 수 (X의 두 번째 차원)
    num_channels = X.shape[1]
    gn = nn.GroupNorm(num_groups, num_channels)
    return gn(X)

# PyTorch GroupNorm 적용
gn_pytorch = group_norm_pytorch(X)
print("\nPyTorch GroupNorm 결과:")
print(gn_pytorch)


PyTorch GroupNorm 결과:
tensor([[[[-1.5275, -1.0911],
          [-0.6547, -0.2182]],

         [[ 0.2182,  0.6547],
          [ 1.0911,  1.5275]],

         [[-1.5275, -1.0911],
          [-0.6547, -0.2182]],

         [[ 0.2182,  0.6547],
          [ 1.0911,  1.5275]]],


        [[[-1.5275, -1.0911],
          [-0.6547, -0.2182]],

         [[ 0.2182,  0.6547],
          [ 1.0911,  1.5275]],

         [[-1.5275, -1.0911],
          [-0.6547, -0.2182]],

         [[ 0.2182,  0.6547],
          [ 1.0911,  1.5275]]]], grad_fn=<NativeGroupNormBackward0>)


In [11]:
# 수동 구현 GroupNorm
def group_norm_manual(X, num_groups=2, eps=1e-5):
    N, C, H, W = X.shape
    G = num_groups
    
    # 그룹별로 텐서 재구성
    X = X.view(N, G, C // G, H, W)
    
    # 평균과 분산 계산
    mean = X.mean(dim=(2, 3, 4), keepdim=True)
    var = X.var(dim=(2, 3, 4), keepdim=True, unbiased=False)
    
    # 정규화
    X = (X - mean) / torch.sqrt(var + eps)
    
    # 원래 shape으로 복원
    return X.view(N, C, H, W)

# 수동 구현 GroupNorm 적용
gn_manual = group_norm_manual(X)
print("\n수동 구현 GroupNorm 결과:")
print(gn_manual)


수동 구현 GroupNorm 결과:
tensor([[[[-1.5275, -1.0911],
          [-0.6547, -0.2182]],

         [[ 0.2182,  0.6547],
          [ 1.0911,  1.5275]],

         [[-1.5275, -1.0911],
          [-0.6547, -0.2182]],

         [[ 0.2182,  0.6547],
          [ 1.0911,  1.5275]]],


        [[[-1.5275, -1.0911],
          [-0.6547, -0.2182]],

         [[ 0.2182,  0.6547],
          [ 1.0911,  1.5275]],

         [[-1.5275, -1.0911],
          [-0.6547, -0.2182]],

         [[ 0.2182,  0.6547],
          [ 1.0911,  1.5275]]]])
